In [13]:
from datasets import load_dataset, concatenate_datasets
import pandas as pd
from sklearn.utils import resample
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
import torch

In [2]:
dataset = load_dataset("financial_phrasebank", 'sentences_50agree')

Dataset financial_phrasebank downloaded and prepared to C:/Users/Gusso/.cache/huggingface/datasets/financial_phrasebank/sentences_50agree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 83.32it/s]


In [3]:
value_count_0=dataset['train']['label'].count(0)
value_count_1=dataset['train']['label'].count(1)
value_count_2=dataset['train']['label'].count(2)


print('number of negatives',value_count_0)
print('number of neutral',value_count_1)
print('number of positives',value_count_2)
#count_1=value_count['1'] # if it is str

number of negatives 604
number of neutral 2879
number of positives 1363


In [15]:

neutrals = dataset.filter(lambda example: example["label"] == 1)
positives = dataset.filter(lambda example: example["label"] == 2)
negatives = dataset.filter(lambda example: example["label"] == 0)

short_positives = positives['train'].select(range(len(negatives['train'])))

print(short_positives)

downsized_dataset = concatenate_datasets([short_positives, negatives['train']])

print(downsized_dataset)

Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-4069f8bdc2072a81.arrow
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-255302c37e6b94c9.arrow
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-913aca77f5bc0114.arrow


Dataset({
    features: ['sentence', 'label'],
    num_rows: 604
})
Dataset({
    features: ['sentence', 'label'],
    num_rows: 1208
})


In [17]:
def replace_values(example):
    # Replace values in the label column
    if "label" in example:
        example["label"] = 1 if example["label"] == 2 else example["label"]
    return example

# Use the map function to apply the custom mapping
downsized_dataset = downsized_dataset.map(replace_values)
downsized_dataset = downsized_dataset.train_test_split(test_size=0.1, shuffle=True)
print(downsized_dataset)


Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-005d6f20e6ad7c92.arrow


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 1087
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 121
    })
})


In [22]:
tokenizer = AutoTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding='max_length', truncation=True, max_length=128)

tokenized_datasets = downsized_dataset['train'].map(tokenize_function, batched=True)
tokenized_datasets_eval = downsized_dataset['test'].map(tokenize_function, batched=True)

In [23]:
model = AutoModelForSequenceClassification.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")


metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets_eval,
    compute_metrics=compute_metrics,
)

In [26]:
trainer.train()

 20%|██        | 83/408 [01:03<04:08,  1.31it/s]
                                                 
 33%|███▎      | 136/408 [00:40<01:16,  3.56it/s]

{'eval_loss': 0.2037314474582672, 'eval_accuracy': 0.9421487603305785, 'eval_runtime': 1.8825, 'eval_samples_per_second': 64.276, 'eval_steps_per_second': 8.499, 'epoch': 1.0}


                                                 
 67%|██████▋   | 272/408 [01:21<00:37,  3.61it/s]

{'eval_loss': 0.22784696519374847, 'eval_accuracy': 0.9504132231404959, 'eval_runtime': 1.855, 'eval_samples_per_second': 65.229, 'eval_steps_per_second': 8.625, 'epoch': 2.0}


                                                 
100%|██████████| 408/408 [02:02<00:00,  3.32it/s]

{'eval_loss': 0.22509229183197021, 'eval_accuracy': 0.9504132231404959, 'eval_runtime': 1.8455, 'eval_samples_per_second': 65.564, 'eval_steps_per_second': 8.67, 'epoch': 3.0}
{'train_runtime': 122.9653, 'train_samples_per_second': 26.52, 'train_steps_per_second': 3.318, 'train_loss': 0.13773220660639743, 'epoch': 3.0}


TrainOutput(global_step=408, training_loss=0.13773220660639743, metrics={'train_runtime': 122.9653, 'train_samples_per_second': 26.52, 'train_steps_per_second': 3.318, 'train_loss': 0.13773220660639743, 'epoch': 3.0})

In [29]:
validation_dataset = load_dataset('zeroshot/twitter-financial-news-sentiment')

def filter_function(example):
    return example["label"] != 2

# Use the filter method to remove records
filtered_dataset = validation_dataset.filter(filter_function)

def tokenize_function2(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True, max_length=128)

tokenized_datasets_validation = filtered_dataset.map(tokenize_function2, batched=True)

# Print the modified dataset
print(dataset)

Found cached dataset csv (C:/Users/Gusso/.cache/huggingface/datasets/zeroshot___csv/zeroshot--twitter-financial-news-sentiment-ccca0f3c622c5b67/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 2/2 [00:00<00:00, 1001.03it/s]
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\zeroshot___csv\zeroshot--twitter-financial-news-sentiment-ccca0f3c622c5b67\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-b2d98c45395bc49c.arrow
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\zeroshot___csv\zeroshot--twitter-financial-news-sentiment-ccca0f3c622c5b67\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-98a6e9e55fa382e5.arrow
                                                                  

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 4846
    })
})


In [30]:
trainer.evaluate(eval_dataset=tokenized_datasets_validation['validation'])

100%|██████████| 103/103 [00:12<00:00,  8.09it/s]


{'eval_loss': 0.9002207517623901,
 'eval_accuracy': 0.791970802919708,
 'eval_runtime': 12.8695,
 'eval_samples_per_second': 63.872,
 'eval_steps_per_second': 8.003,
 'epoch': 3.0}